In [16]:
import traceback
# from csv import writer
from csv import DictWriter
import pandas as pd
# from sqlalchemy import create_engine, text
from datetime import datetime, date, timedelta
import time
import config
from datetime import timezone
from polygon import StocksClient
from polygon import ReferenceClient
from typing import Iterator, Optional
import io
import asyncio
# import asyncpg
import json
from itertools import dropwhile
# import uvloop  # Unix only
# import nautilus_trader
# from nautilus_trader.model.data.tick import QuoteTick
# from nautilus_trader.model.objects import Price, Quantity
# from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.backtest.data.providers import TestInstrumentProvider
from nautilus_trader.model.currencies import USD
from nautilus_trader.model.currency import Currency
from nautilus_trader.model.enums import AssetClass, AggregationSource, BarAggregation, PriceType
from nautilus_trader.model.enums import OptionKind
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.identifiers import Symbol
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.instruments.currency_pair import CurrencyPair
from nautilus_trader.model.instruments.equity import Equity
from nautilus_trader.model.instruments.future import Future
from nautilus_trader.model.instruments.option import Option
from nautilus_trader.model.objects import Money
from nautilus_trader.model.objects import Price
from nautilus_trader.model.objects import Quantity
from nautilus_trader.persistence.external.core import write_objects
from nautilus_trader.model.data.bar import BarType, BarSpecification
from nautilus_trader.backtest.data.wranglers import BarDataWrangler

In [17]:
stock_cols = ['symbol', 'tick_volume', 'total_volume', 'opening_price', 'tick_vwap', 'tick_open', 'tick_close', 'tick_high', 'tick_low', 'vwap', 'avg_trade_size', 'time_beg', 'time_end', 'tdate', 'save_date']
key = config.polygon_key
symbol = ['TSLA']

In [18]:
symbol
#await stocks_client.close()

['TSLA']

In [19]:
catalog_path = 'C:\Repos\polygon_nautilus\.data\polygon_nautilus'

# Create an instance of the ParquetDataCatalog
%time catalog = ParquetDataCatalog(catalog_path)

CPU times: total: 0 ns
Wall time: 0 ns


In [20]:
%time ref_client = ReferenceClient(key, False)
%time tickers = ref_client.get_tickers(market='stocks', all_pages=True, merge_all_pages=True)
%time ref_client.close

CPU times: total: 0 ns
Wall time: 0 ns
CPU times: total: 46.9 ms
Wall time: 2.72 s
CPU times: total: 0 ns
Wall time: 0 ns


<bound method BaseClient.close of <polygon.reference_apis.reference_api.SyncReferenceClient object at 0x0000015877287D90>>

In [21]:
%time df = pd.DataFrame.from_dict(tickers)
%time df.cik = df['cik'].astype(str)
%time df = df[df['ticker'].isin(symbol)]

CPU times: total: 0 ns
Wall time: 29 ms
CPU times: total: 0 ns
Wall time: 5.08 ms
CPU times: total: 0 ns
Wall time: 6 ms


In [22]:
df['ticker']

10889    TSLA
Name: ticker, dtype: object

In [23]:
def generic_equity(ticker_symbol: str, ticker_venue: Venue, currency, cik) -> Equity:
    return Equity(
        instrument_id=InstrumentId(symbol=Symbol(ticker_symbol), venue=Venue(ticker_venue)),
        native_symbol=Symbol(ticker_symbol),
        currency=USD,
        price_precision=2,
        price_increment=Price.from_str("0.01"),
        multiplier=Quantity.from_int(1),
        lot_size=Quantity.from_int(1),
        isin=cik,  # we should be converting this to isin...
        ts_event=0,
        ts_init=0,
    )

In [24]:
for index, row in df.iterrows():
    %time instrument = generic_equity(row.ticker, row.primary_exchange, row.currency_name, row.cik)

CPU times: total: 0 ns
Wall time: 0 ns


In [25]:
instrument

Equity(id=TSLA.XNAS, native_symbol=TSLA, asset_class=EQUITY, asset_type=SPOT, quote_currency=USD, is_inverse=False, price_precision=2, price_increment=0.01, size_precision=0, size_increment=1, multiplier=1, lot_size=1, margin_init=0, margin_maint=0, maker_fee=0, taker_fee=0, info={})

In [26]:
%time write_objects(catalog, [instrument])

CPU times: total: 15.6 ms
Wall time: 37.7 ms


In [27]:
start_date = date(2021, 1, 1)
end_date = date(2021, 1, 31)

In [28]:
start_date, end_date

(datetime.date(2021, 1, 1), datetime.date(2021, 1, 31))

In [ ]:
%time stocks_client = StocksClient(key, use_async=True, read_timeout=60)  # Async client

In [ ]:
async def get_bar_data(ticker: str, start_date, end_date):
    resp = await stocks_client.get_aggregate_bars(symbol=ticker
                                            , from_date=start_date, to_date=end_date, timespan='minute'
                                            , full_range=True, run_parallel=True, max_concurrent_workers=8
                                            , high_volatility=True, warnings=False, adjusted=True)

    for d in resp:
        d.setdefault('a')
        d.setdefault('op')
        d.setdefault('n')
        d.setdefault('vw')
        if 'a' in d:
            del d['a']
        if 'n' in d:
            del d['n']
        if 'op' in d:
            del d['op']
    resp = [{'v': d['v'], 'vw': d['vw'], 'o': d['o'], 'c': d['c'], 'h': d['h'], 'l': d['l'], 't': d['t']} for d in resp]
    for d in resp:
        d['bar_volume'] = d.pop('v')
        d['bar_vwap'] = d.pop('vw')
        d['bar_open'] = d.pop('o')
        d['bar_close'] = d.pop('c')
        d['bar_high'] = d.pop('h')
        d['bar_low'] = d.pop('l')
        d['bar_time_start'] = d.pop('t')
        d['symbol'] = ticker
        d['bar_volume'] = int(d['bar_volume'])
        d['timestamp'] = unix_convert(d['bar_time_start'])  # pd.Timestamp(d['bar_time_start'], tz='UTC')  # pd.Timestamp(datetime.datetime.strptime(d['bar_time_start'].decode(), "%Y%m%d %H%M%S%f"), tz='UTC')
        d['save_date'] = datetime.now(timezone.utc)

    resp = [{'symbol': d['symbol'], 'bar_time_start': d['bar_time_start'], 'open': d['bar_open'], 'high': d['bar_high'],
             'low': d['bar_low'], 'close': d['bar_close'], 'volume': d['bar_volume'], 'vwap': d['bar_vwap'],
             'timestamp': d['timestamp'], 'save_date': d['save_date']
             } for d in resp]

    print('hi')

    return resp

In [ ]:
await get_bar_data(instrument.symbol, start_date, end_date)

In [ ]:
ticker_dict

In [ ]:
nautilus_bars = asyncio.run(get_instrument_data(instrument))
# write bars to catalog

print(f"Done processing ticker {row.ticker}")

In [43]:
ticker_dict

<coroutine object get_bar_data at 0x0000015877332F80>

In [ ]:
nautilus_bars = asyncio.run(get_instrument_data(instrument))
# write bars to catalog

print(f"Done processing ticker {row.ticker}")

In [ ]:
stocks_client = StocksClient(key, use_async=True, read_timeout=60)  # Async client

In [ ]:
try:
    for index, row in df.iterrows():
        instrument = generic_equity(row.ticker, row.primary_exchange, row.currency_name, row.cik)
        write_objects(catalog, [instrument])

        nautilus_bars = asyncio.run(get_instrument_data(instrument))
        # write bars to catalog

        print(f"Done processing ticker {row.ticker}")

except Exception as e:
    print(e)
    traceback.print_exc()

finally:
    await stocks_client.close()